The task is to make an exploratorary data analysis on [given data](../data/raw%20data).

# Plan 

We are given an excel table and a bunch of photos. What we gonna do is:
- parse .xlsx file to DataFrame, so we can conviniently work with it
- parse name from given photos
- draw some plots

# Solution 

In [235]:
import pandas as pd
import numpy as np
import seaborn as sb

Let's start from parsing .xlsx:

## .xlsx parsing 

In [236]:
df = pd.read_excel("../data/raw data/Послойное описание с фотографиями.xlsx")

In [237]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,Месторождение,Скважина,"Интервал отбора керна, м",NaN,"Интервал отбора керна, м (после привязки)",NaN,Фактический вынос,NaN,Стратиграфическая единица,Слой,...,Пустотное пространство,Трещиноватость (морфологические характеристики...,"Нефтенасыщение/ битуминозность (характер, инте...",Коллектор (тип)/не коллектор,Целостность керна,Границы слоя (контакт с нижележащим слоем),Примечание,"Профильные методы,\nсреднее значение по слою",NaN,NaN
1,NaN,NaN,Кровля,Подошва,Кровля,Подошва,м,%,NaN,Кровля,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Общая радио активность, API","Плотность, г/см3","Проницаемость, мД"
2,1,2,3,4,5,6,7,8,9,10,...,29,30,31,32,33,34,35,36,37,38
3,1,2,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.9217,2.41228,2.07915
4,1,2,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.7345,2.48618,0.490632


In [238]:
# Drop unused columns.
df.drop(df.columns[19:35],axis=1,inplace=True)    # Blank columns.
df.drop(df.columns[12:14], axis=1, inplace=True)  # Photos columns.

In [239]:
# Merging together 1st and 2nd rows.
fst_row = list(df.iloc[0])
snd_row = list(df.iloc[1])

# If value from fst row == nan, then replace nan with value from prev value.
for i in range(len(fst_row)):
    if (fst_row[i]!=fst_row[i]):
        fst_row[i] = fst_row[i-1]
    if "\n" in fst_row[i]:
        fst_row[i] = fst_row[i].replace("\n", " ")

# If value from snd row != nan, then add it to fst row.
for i in range(len(fst_row)):
    if (snd_row[i]==snd_row[i]):
        fst_row[i] += " ({})".format(snd_row[i])        

# Replce fst row with received row.
df.iloc[0] = fst_row

# Changing column names.
df.columns = df.iloc[0]

In [240]:
# Dropping 2nd and 3rd rows (3rd row - numeration).
df = df.drop([0,1,2])

# Resetting indexes.
df = df.reset_index(drop=True)

In [241]:
# Changing types to float where possible.
columns = list(df.columns)
columns = columns[0:8]+columns[9:12]+columns[17:20]

df[columns] = df[columns].astype(float)

In [242]:
df.head()

,Месторождение,Скважина,"Интервал отбора керна, м (Кровля)","Интервал отбора керна, м (Подошва)","Интервал отбора керна, м (после привязки) (Кровля)","Интервал отбора керна, м (после привязки) (Подошва)",Фактический вынос (м),Фактический вынос (%),Стратиграфическая единица,Слой (Кровля),Слой (Подошва),Слой (Толщина),Классификация (Порода),Классификация (Карбонатность),Классификация (Разрушенность),Классификация (Насыщение),Полное описание (старый формат),"Профильные методы, среднее значение по слою (Общая радио активность, API)","Профильные методы, среднее значение по слою (Плотность, г/см3)","Профильные методы, среднее значение по слою (Проницаемость, мД)"
0,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,0.00,1.45,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные,"Песчаник светло-серый с коричневатым оттенком,...",65.921720,2.412277,2.079148
1,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,1.45,3.54,2.09,алевролит,не карбонатный,не разрушен,не опред.,Алевролит серый от мелко-крупнозернистого до м...,78.734522,2.486175,0.490632
2,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,3.54,5.85,2.31,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,Песчаник светло-серый мелкозернистый алевритис...,66.105366,2.441883,22.843947
3,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,5.85,5.93,0.08,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные,Тонкое неравномерное переслаивание алевролита ...,64.195236,2.443174,38.450000
4,1.0,2.0,1963.1,1978.1,1964.3,1979.3,15.00,1.000000,НП\2-3,0.00,1.26,1.26,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,Песчаник светло-серый мелко-среднезернистый до...,60.734262,2.458812,22.261125


##  .xlsx analyzing

Now we have all data prepared, so let's take a look on it.

In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 20 columns):
Месторождение                                                                52 non-null float64
Скважина                                                                     52 non-null float64
Интервал отбора керна, м (Кровля)                                            52 non-null float64
Интервал отбора керна, м (Подошва)                                           52 non-null float64
Интервал отбора керна, м (после привязки) (Кровля)                           52 non-null float64
Интервал отбора керна, м (после привязки) (Подошва)                          52 non-null float64
Фактический вынос (м)                                                        52 non-null float64
Фактический вынос (%)                                                        52 non-null float64
Стратиграфическая единица                                                    52 non-null object
Слой (Кровля)            

In [220]:
df.describe()

,Месторождение,Скважина,"Интервал отбора керна, м (Кровля)","Интервал отбора керна, м (Подошва)","Интервал отбора керна, м (после привязки) (Кровля)","Интервал отбора керна, м (после привязки) (Подошва)",Фактический вынос (м),Фактический вынос (%),Слой (Кровля),Слой (Подошва),Слой (Толщина),"Профильные методы, среднее значение по слою (Общая радио активность, API)","Профильные методы, среднее значение по слою (Плотность, г/см3)","Профильные методы, среднее значение по слою (Проницаемость, мД)"
count,52.0,52.0,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,1.0,2.0,2031.368077,2043.483077,2032.654615,2044.769615,12.092692,0.994197,4.381154,5.664808,1.283654,67.079364,2.497749,6.246265
std,0.0,0.0,95.668992,96.291088,95.805202,96.427562,4.448389,0.019489,3.726577,4.057599,1.503611,8.670555,0.056785,8.989631
min,1.0,2.0,1957.100000,1963.100000,1958.300000,1964.300000,2.880000,0.917197,0.000000,0.310000,0.080000,47.510938,2.338011,0.010000
25%,1.0,2.0,1963.100000,1978.100000,1964.300000,1979.300000,6.735000,1.000000,1.420000,2.537500,0.325000,60.673190,2.460604,0.371021
50%,1.0,2.0,1979.670000,1988.750000,1980.870000,1989.950000,14.960000,1.000000,4.130000,5.000000,0.760000,66.340135,2.500482,1.727099
75%,1.0,2.0,2176.060000,2191.020000,2177.560000,2192.520000,15.000000,1.000000,5.772500,7.002500,1.510000,71.535949,2.536002,7.159025
max,1.0,2.0,2191.020000,2198.000000,2192.520000,2199.500000,15.000000,1.000000,14.440000,15.000000,7.930000,87.314407,2.610877,38.450000


In [222]:
df.head()

,Месторождение,Скважина,"Интервал отбора керна, м (Кровля)","Интервал отбора керна, м (Подошва)","Интервал отбора керна, м (после привязки) (Кровля)","Интервал отбора керна, м (после привязки) (Подошва)",Фактический вынос (м),Фактический вынос (%),Стратиграфическая единица,Слой (Кровля),Слой (Подошва),Слой (Толщина),Классификация (Порода),Классификация (Карбонатность),Классификация (Разрушенность),Классификация (Насыщение),Полное описание (старый формат),"Профильные методы, среднее значение по слою (Общая радио активность, API)","Профильные методы, среднее значение по слою (Плотность, г/см3)","Профильные методы, среднее значение по слою (Проницаемость, мД)"
0,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,0.00,1.45,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные,"Песчаник светло-серый с коричневатым оттенком,...",65.921720,2.412277,2.079148
1,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,1.45,3.54,2.09,алевролит,не карбонатный,не разрушен,не опред.,Алевролит серый от мелко-крупнозернистого до м...,78.734522,2.486175,0.490632
2,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,3.54,5.85,2.31,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,Песчаник светло-серый мелкозернистый алевритис...,66.105366,2.441883,22.843947
3,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.988333,НП\2-3,5.85,5.93,0.08,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные,Тонкое неравномерное переслаивание алевролита ...,64.195236,2.443174,38.450000
4,1.0,2.0,1963.1,1978.1,1964.3,1979.3,15.00,1.000000,НП\2-3,0.00,1.26,1.26,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,Песчаник светло-серый мелко-среднезернистый до...,60.734262,2.458812,22.261125


In [231]:
df["Классификация (Порода)"].value_counts()

песчаник                                35
пересл. песчаника, алевролита и глин    10
алевролит                                7
Name: Классификация (Порода), dtype: int64

In [232]:
df["Классификация (Карбонатность)"].value_counts()

не карбонатный                              35
сильнокарбонатный                            6
с карбонатными обломками или конкрециями     6
среднекарбонатный                            3
слабокарбонатный                             2
Name: Классификация (Карбонатность), dtype: int64

In [233]:
df["Классификация (Разрушенность)"].value_counts()

не разрушен    52
Name: Классификация (Разрушенность), dtype: int64

In [234]:
df["Классификация (Насыщение)"].value_counts()

слабо нефтенасыщенные       17
пятнисто нефтенасыщенные    16
не опред.                   15
нефтенасыщенные              4
Name: Классификация (Насыщение), dtype: int64

In [243]:
df["Фактический вынос (м)"].value_counts()

15.00    24
14.96    12
6.00      6
5.93      4
6.98      3
2.88      3
Name: Фактический вынос (м), dtype: int64